# Q4_Optimization

运行 TwoDimDiffusion.zip 中的程序，体会、理解和掌握有限差分和有限体积法的求解 2D 热传导方程的基本步骤。掌握输出给定点温度值的方法

本文关于 alpha 求值部分存在不完善的地方，建议只参考算法优化部分，如需查看正确求值步骤，建议阅读 Q4_Answer

由例程可知，本题问题实质为左侧向右侧传热的过程，而例程中的方法为有限差分法

对于一维的有限差分法，我们由 $Q2(2.5)(2.8)$ 可知：

$$T^{t+\Delta t}_i=\alpha (T^t_{i+1}-2T^t_i+T^t_{i-1})+T^t_i\tag{0.1}$$

$$T^{t+\Delta t}_i = T^t_i + \alpha_e(i) \cdot (T^t_{i+1}-T^t_i) + \alpha_w(i) \cdot (T^t_{i-1}-T^t_i)\tag{0.2}$$

而二维的有限差分法与一维类似：

$$T^{t+\Delta t}_{i,j} = T^t_{i,j} + \alpha_e(i,j) \cdot (T^t_{i+1,j}-T^t_{i,j}) + \alpha_w(i,j) \cdot (T^t_{i-1,j}-T^t_{i,j})+\alpha_s(i,j) \cdot (T^t_{i,j-1}-T^t_{i,j}) + \alpha_n(i,j) \cdot (T^t_{i,j+1}-T^t_{i,j}) \tag{0.3}$$

$$T^{t+\Delta t}_{i,j} = T^t_{i,j} + \alpha(i,j) \cdot (T^t_{i+1,j}+T^t_{i-1,j}+T^t_{i,j-1}+T^t_{i,j+1}-4T^t_{i,j})\tag{0.4}$$

变换成代码就是：

```python
# 一维温度计算
tlist_new[i] = tlist_old[i] + alpha * \
    (tlist_old[i + 1] + tlist_old[i - 1] -
     2 * tlist_old[i])

# 二维温度计算
tlist_new[row][col] = tlist_old[row][col] + alpha * \
    (tlist_old[row+1][col] + tlist_old[row-1][col] +
     tlist_old[row][col+1] + tlist_old[row][col-1] -
     4 * tlist_old[row][col])
```

## 第一步：初始化相关参数

In [1]:
col_len = 30  # 列数
row_len = 50  # 行数
node_num = col_len * row_len  # 网格数量

t_left = 100.0  # 左边界侧温度

max_loops = 200000  # 迭代次数
step_size = 2000  # 保存步长

In [2]:
thermal_conductivity = 237  # 导热系数 (W/(m·K)) 铝导热系数为 80
density = 2700  # 密度 (kg/m^3) 铝密度为 7800
specific_heat_capacity = 880  # 比热容 (J/kg*℃)

由$Q2(1.2)(1.4)$ 得：

$$\alpha = \frac{\lambda \Delta t}{\rho C_p (\Delta x)^2}\tag{1.1}$$

$$\Delta t = 0.5 \cdot \frac{\rho C_p \Delta x^2}{2\lambda}\tag{1.2}$$

In [3]:
delta_x = 0.005  # 网格长度 (m)

# 这里为了多跑几个 loop 特地把 0.5 改成了 0.002
delta_t = 0.002*(density*specific_heat_capacity*pow(delta_x, 2))\
    / (2*thermal_conductivity)
print("delta_t: ", delta_t, "s")

alpha = (thermal_conductivity*delta_t)\
    / (density*specific_heat_capacity*pow(delta_x, 2))
print("alpha:", alpha)

delta_t:  0.0002506329113924051 s
alpha: 0.001


## 第二步：定义有限差分算法

### 1. 传统算法

传统算法，即使用两个 `for` 循环一个个遍历数组，从而计算对于节点的新温度

对于传统算法，对应的数据结构使用二维数组 `List[][]` 即可

In [4]:
tlist_old_1 = [[20.0]*col_len for row in range(row_len)]  # 记录旧温度
tlist_new_1 = [[20.0]*col_len for row in range(row_len)]  # 记录新温度

In [5]:
def thermal_conductivity_1(tlist_new, tlist_old):
    
    # 忽略下边界计算，避免数组越界，计算边界是因为在 python 中，[-1] 就指向最后位
    for row in range(row_len-1):
        # 忽略左右边界计算，
        # 一是避免数组越界，
        # 二是下文会重新计算左右边界，因此简化计算步骤
        for col in range(1, col_len-1):
            tlist_new[row][col] = tlist_old[row][col] + alpha * \
                (tlist_old[row][col+1]+tlist_old[row][col-1] + tlist_old[row+1]
                      [col]+tlist_old[row-1][col] - 4 * tlist_old[row][col])
            
    # 下边界计算
    for col in range(1, col_len-1):
        tlist_new[-1][col] = tlist_old[-1][col] + alpha * \
            (tlist_old[-1][col+1]+tlist_old[-1][col-1] + tlist_old[0]
              [col]+tlist_old[-2][col] - 4 * tlist_old[-1][col])

    # 计算左边界
    for row in range(row_len):
        tlist_new[row][0] = t_left

    # 计算右边界 partial T / partial x = 0
    for row in range(row_len):
        tlist_new[row][-1] = tlist_new[row][-2]

    return tlist_new

### 2. 向量法

不用算，我们都可以知道传统算法的效率并不高，毕竟时间复杂度 $O(n^2)$ 摆在那呢

而且再怎么优化提升也不大，那么如果我们对数据换个处理方法呢？

在传统算法中，我们一次处理一个数据，如果我们处理一行数据呢？

要对“向量”进行操作，我们就需要借助 python 的一个数学库 `NumPy`

安装 `NumPy` 很简单，我们只要执行以下命令即可：

```bash
pip3 install numpy
```

Linux 用户如果提示权限不足，请在命令前面添加 `sudo`

In [6]:
import numpy as np  # 导入 numpy 库

tlist_new_2 = np.empty([row_len, col_len])  # 记录新温度，这里是创建一个空数组
tlist_old_2 = np.array([[20.0] * col_len for row in range(row_len)])  # 记录旧温度

既然我们用了向量法，那么热传导函数也需要进行相应的修改：

```python
tlist_new[row][col] = tlist_old[row][col] + alpha * \
    (tlist_old[row+1][col] + tlist_old[row-1][col] +
     tlist_old[row][col+1] + tlist_old[row][col-1] -
     4 * tlist_old[row][col])
```

我们将行运算和列运算拆分，可变换为：

```python
tlist_new[row][col] = tlist_old[row][col] + \
    alpha * (tlist_old[row+1][col] + tlist_old[row-1][col] -
             2 * tlist_old[row][col]) + \
    alpha * (tlist_old[row][col+1] + tlist_old[row][col-1] -
             2 * tlist_old[row][col])
```

可以看到这里每个节点关于 `alpha` 的乘法需要进行两次

而乘法是非常消耗资源，我们应该尽可能减少乘法计算才行，即集中处理乘法计算

那么可进一步变换为：

```python
tlist_new[row][col] = tlist_old[row+1][col] + tlist_old[row-1][col] - \
    2 * tlist_old[row][col]
tlist_new[row][col] += tlist_old[row][col+1] + tlist_old[row][col-1] - \
    2 * tlist_old[row][col]
tlist_new[row][col] = alpha * tlist_new[row][col] + tlist_old[row][col]
```

In [7]:
def thermal_conductivity_2(tlist_new, tlist_old):

    # 先对行计算
    for row in range(row_len-1):  # 去下边界
        tlist_new[row] = tlist_old[row-1] + \
            tlist_old[row+1] - 2 * tlist_old[row]

    # 下边界行计算
    tlist_new[-1] = tlist_new[-1] + tlist_old[-2] + \
        tlist_old[0] - 2 * tlist_old[-1]

    # 再对列计算
    for col in range(1, col_len-1):  # 去 左右 边界
        tlist_new[:, col] = tlist_old[:, col-1] + \
            tlist_old[:, col+1] - 2 * tlist_old[:, col]

    tlist_new = alpha * tlist_new + tlist_old

    tlist_new[:, 0] = t_left  # 左边界
    tlist_new[:, -1] = tlist_new[:, -2]  # 右边界: partial T / partial x = 0

    return tlist_new

### 3. 向量法 + Numba 库

向量法已经很快了，那么还有没有更快的方法呢？

我们都知道 Python 比较慢，这是由于 Python 太动态了

如果能事先编译一下 Python，让它静态一点，速度应该就会上来

于是我们就有了 `Cython`，然而 `Cython` 毕竟不是原生的 Python 代码，使用起来还是有诸多不便的

这里我们使用到了 `Numba` 库，它通过提供一些装饰器，从而将其修饰的函数编译成机器码函数，并返回一个可以在 Python 中调用机器码的包装对象

所以我们就可以再次提速啦~

In [8]:
tlist_new_3 = np.empty([row_len, col_len])  # 记录新温度，这里是创建一个空数组
tlist_old_3 = np.array([[20.0] * col_len for row in range(row_len)])  # 记录旧温度

In [9]:
from numba import jit  # 导入 numba 库


@jit(nopython=True)  # JIT 装饰器
def thermal_conductivity_3(tlist_new, tlist_old):

    # 先对行计算
    for row in range(row_len-1):  # 去下边界
        tlist_new[row] = tlist_old[row-1] + \
            tlist_old[row+1] - 2 * tlist_old[row]

    # 下边界行计算
    tlist_new[-1] = tlist_new[-1] + tlist_old[-2] + \
        tlist_old[0] - 2 * tlist_old[-1]

    # 再对列计算
    for col in range(1, col_len-1):  # 去 左右 边界
        tlist_new[:, col] = tlist_old[:, col-1] + \
            tlist_old[:, col+1] - 2 * tlist_old[:, col]

    # 赋予权重并求和
    tlist_new = alpha * tlist_new + tlist_old

    tlist_new[:, 0] = t_left  # 左边界
    tlist_new[:, -1] = tlist_new[:, -2]  # 右边界: partial T / partial x = 0

    return tlist_new

### 4. 矩阵法 + Numba 库

我们知道向量法 + `Numba` 库真的已经非常非常的快了

那如果把向量法换成矩阵法呢？一次处理一个矩阵，岂不美滋滋？

使用矩阵法最重要的一点就是不超界

因此在求四周差值时，我们需要将矩阵去边，即四周大小均缩小 1 点，再使用矩阵加法来计算

其过程类似于抖动矩阵归一化操作，或者是深度学习中的池化操作

In [10]:
tlist_new_4 = np.empty([row_len, col_len])  # 记录新温度，这里是创建一个空数组
tlist_old_4 = np.array([[20.0] * col_len for row in range(row_len)])  # 记录旧温度

In [11]:
@jit(nopython=True)  # JIT 装饰器
def thermal_conductivity_4(tlist_new, tlist_old):

    # 先计算去边矩阵
    tlist_new[1:-1, 1:-1] = alpha * (tlist_old[:-2, 1:-1] + tlist_old[2:, 1:-1] +
                                             tlist_old[1:-1, :-2] + tlist_old[1:-1, 2:] -
                                             4 * tlist_old[1:-1, 1:-1]) + tlist_old[1:-1, 1:-1]

    # 上边界上下左右抖动计算差值
    tlist_new[0, 1:-1] = alpha * (tlist_old[1, 1:-1] + tlist_old[-1, 1:-1] +
                                          tlist_old[0, :-2] + tlist_old[0, 2:] -
                                          4 * tlist_old[0, 1:-1]) + tlist_old[0, 1:-1]

    # 下边界上下左右抖动计算差值
    tlist_new[-1, 1:-1] = alpha * (tlist_old[-2, 1:-1] + tlist_old[0, 1:-1] +
                                           tlist_old[-1, :-2] + tlist_old[-1, 2:] -
                                           4 * tlist_old[-1, 1:-1]) + tlist_old[-1, 1:-1]

    tlist_new[:, 0] = t_left  # 左边界
    tlist_new[:, -1] = tlist_new[:, -2]  # 右边界: partial T / partial x = 0

    return tlist_new

## 第三步：启动循环

让我们来跑个分，看看谁最快~

In [12]:
import time  # 导入时间库

### 1. 传统算法

In [13]:
import copy
t1 = time.time()
thermal_data_1 = []
append = thermal_data_1.append
for loops in range(max_loops+1):
    tlist_old_1 = copy.deepcopy(thermal_conductivity_1(tlist_new_1, tlist_old_1))  # 深拷贝
    if loops % step_size == 0:
        append(tlist_old_1)
t2 = time.time()
print("耗时：", t2-t1, "s")

耗时： 382.9026267528534 s


### 2. 向量法

In [14]:
t1 = time.time()
thermal_data_2 = []
append = thermal_data_2.append
for loops in range(max_loops+1):
    tlist_old_2 = thermal_conductivity_2(tlist_new_2, tlist_old_2)
    if loops % step_size == 0:
        append(tlist_old_2)
t2 = time.time()
print("耗时：", t2-t1, "s")

耗时： 57.75068211555481 s


### 3. 向量法 + Numba 库

In [15]:
t1 = time.time()
thermal_data_3 = []
append = thermal_data_3.append
for loops in range(max_loops+1):
    tlist_old_3 = thermal_conductivity_3(tlist_new_3, tlist_old_3)
    if loops % step_size == 0:
        append(tlist_old_3)
t2 = time.time()
print("耗时：", t2-t1, "s")

耗时： 5.663832664489746 s


### 4. 矩阵法 + Numba 库

In [16]:
t1 = time.time()
thermal_data_4 = []
append = thermal_data_4.append
for loops in range(max_loops+1):
    tlist_old_4 = thermal_conductivity_4(tlist_new_4, tlist_old_4).copy()  # 深拷贝
    if loops % step_size == 0:
        append(tlist_old_4)
t2 = time.time()
print("耗时：", t2-t1, "s")

耗时： 4.123006582260132 s


### 耗时分析

经过漫长的等待，结果出来了：

1. 传统双 for 算法耗时 382.9s

2. 使用了 numpy 库的向量法耗时 57.8s

3. 加持了 numba 的向量法耗时 5.7s

4. 加持了 numba 的矩阵法耗时 4.1s

可以看到，经过一通优化，最优算法相较于传统算法提速 93 倍，提速效果较为理想

同时代码相比于传统算法更为易简洁易懂

注：

> 1. 为了消除平均误差，每个算法均运行 20w 个 loop
>
> 2. 此处所说的最优算法仅表示个人看法，如有大佬，欢迎 pr 指正
>
> 3. 使用 numba 库还可以调用并行加速，以及 CUDA 加速，此处并未涉及

可见没有运行慢的语言，只有运行慢的代码 XD

## 第四步：绘图对比

Function 1-4 分别对应四种算法

为了更快的预览交互式图片，建议连接 vpn

In [17]:
# 导入绘图库
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [18]:
data = [go.Heatmap(visible=False, zmin=20, zmax=100, z=thermal_data_1[-1],
                   hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> "),
       go.Heatmap(visible=False, zmin=20, zmax=100, z=thermal_data_2[-1],
                   hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> "),
       go.Heatmap(visible=False, zmin=20, zmax=100, z=thermal_data_3[-1],
                   hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> "),
       go.Heatmap(visible=False, zmin=20, zmax=100, z=thermal_data_4[-1],
                   hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ")]

data[0]['visible'] = True

In [20]:
steps = []
data_len = len(data)
for i in range(data_len):
    step = dict(
        method='restyle',
        args=['visible', [False] * data_len],
        label=i
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Function: "},
    steps=steps
)]

layout = dict(sliders=sliders, width = 800, height = 800)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig)